In [2]:
#imports 
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import os
import locale
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input, Add, ZeroPadding2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from sklearn import preprocessing
import locale
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

#CNN involving only the first three blocks of the ResNet
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1',)(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')         (X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def create_cnn(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', )(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    headModel = Flatten()(X)
    headModel=Dense(256, activation='relu', name='fc1')(headModel)
    headModel=Dense(128, activation='relu', name='fc2')(headModel)
    headModel = Dense( 1, name='fc3')(headModel)
    model = Model(inputs=X_input, outputs=headModel, name='ResNet50')
    return model

gasdata = pd.read_csv('/Users/eddie/Downloads/TrainGasSensorMeasurements.csv')
data_dir = '/Users/eddie/Downloads/NoGas/'
#load the image files
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        print(image)
        img = mpimg.imread('/Users/eddie/Downloads/NoGas/'+image)
        img.resize(224,224,3)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
#store loaded images

#split images for training and testing 
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
#normalize gas sensor data
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)

#train the CNN 
model = create_cnn()
from tensorflow.keras.models import load_model
model = load_model('/Users/eddie/Downloads/ModifiedDFC/CNNs/ResNet50.keras')
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001), loss='binary_crossentropy',metrics = ['acc'])
hist = model.fit(trainImagesX,trainy,validation_data=(testImagesX,testy),epochs=25,batch_size = 20)

model.save('/Users/eddie/Downloads/ModifiedDFC/CNNs/ResNet50.keras')





6400
A (1).png
A (10).png
A (100).png
A (1000).png
A (1001).png
A (1002).png
A (1003).png
A (1004).png
A (1005).png
A (1006).png
A (1007).png
A (1008).png
A (1009).png
A (101).png
A (1010).png
A (1011).png
A (1012).png
A (1013).png
A (1014).png
A (1015).png
A (1016).png
A (1017).png
A (1018).png
A (1019).png
A (102).png
A (1020).png
A (1021).png
A (1022).png
A (1023).png
A (1024).png
A (1025).png
A (1026).png
A (1027).png
A (1028).png
A (1029).png
A (103).png
A (1030).png
A (1031).png
A (1032).png
A (1033).png
A (1034).png
A (1035).png
A (1036).png
A (1037).png
A (1038).png
A (1039).png
A (104).png
A (1040).png
A (1041).png
A (1042).png
A (1043).png
A (1044).png
A (1045).png
A (1046).png
A (1047).png
A (1048).png
A (1049).png
A (105).png
A (1050).png
A (1051).png
A (1052).png
A (1053).png
A (1054).png
A (1055).png
A (1056).png
A (1057).png
A (1058).png
A (1059).png
A (106).png
A (1060).png
A (1061).png
A (1062).png
A (1063).png
A (1064).png
A (1065).png
A (1066).png
A (1067).png
A (106

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/Users/eddie/Downloads/ModifiedDFC/CNNs/ResNetWeights.keras')
#evaluate performance
performance = model.predict(testImagesX)
performance.round()
actual = []
for value in performance: 
    #print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)

acutal = np.array(actual)
print(gasdata)
accuracy = accuracy_score(testy,actual)
print('Accuracy: %f' % accuracy)
precision = precision_score(testy, actual)
print('Precision: %f' % precision)
recall = recall_score(testy, actual)
print('Recall: %f' % recall)
f1 = f1_score(testy, actual)
print('F1 score: %f' % f1)

NameError: name 'testImagesX' is not defined

In [ ]:
import time
import psutil
import matplotlib.image as mpimg
from tensorflow.keras.models import load_model
model = load_model('/Users/eddie/Downloads/ModifiedDFC/CNNs/ResNetWeights.keras')

def measure_latency_cpu_usage(model, test_inputs):
    process = psutil.Process()
    cpu_start = process.cpu_percent()
    mem_start = process.memory_info()
    start = time.time()
    predictions = model.predict(test_inputs)
    end = time.time()
    cpu_end = process.cpu_percent()
    latency = end - start
    cpu_usage = cpu_end - cpu_start
    memory = process.memory_info()
    return latency, cpu_usage, memory.rss/(1024**2)-mem_start.rss/(1024**2)

# Example usage:

testImagesX = mpimg.imread("/Users/eddie/Downloads/NoGas/D (94).png")
testImagesX.resize(224,224,3)
testImagesX = testImagesX/255.0
testImagesX = testImagesX.reshape(1,224,224,3)
latency, cpu_usage, memory = measure_latency_cpu_usage(model, testImagesX)

In [ ]:
print(latency, memory)

0.5533745288848877 46.140625
